## 分类（矩形是胖的还是瘦的）

## 1. 生成数据

In [5]:
# 生成矩形数据的函数
def get_rectangle():
    import random
    width, height = random.random(), random.random()
    fat = int(width >= height)
    return width, height, fat

get_rectangle() 

(0.41554273917008755, 0.15227609329823955, 1)

## 2. 定义数据集

In [6]:
import torch

class Dataset(torch.utils.data.Dataset):
    
    #正常应该在这里执行数据的加载，处理等操作
    def __init__(self):
        pass
    
    #定义数据的条数
    def __len__(self):
        return 10000
    
    #根据序号i，获取数据
    def __getitem__(self, i):
        width, height, fat = get_rectangle()
        x = torch.FloatTensor([width, height])
        y = fat
        return x, y
    
dataset = Dataset()

len(dataset), dataset[0]

(10000, (tensor([0.2583, 0.8427]), 0))

## 3. 定义Loader 数据遍历器

In [7]:
#数据集加载器，每8条数据组成一个batch，打乱顺序，不足的舍弃
loader = torch.utils.data.DataLoader(dataset=dataset, batch_size=8, shuffle=True, drop_last=True)
(len(loader), next(iter(loader))) #查看loader的长度，以及第一个batch的数据

(1250,
 [tensor([[0.2350, 0.5281],
          [0.4245, 0.1665],
          [0.8516, 0.7542],
          [0.5544, 0.7940],
          [0.3912, 0.7144],
          [0.8413, 0.9484],
          [0.8172, 0.4960],
          [0.8866, 0.4358]]),
  tensor([0, 1, 1, 0, 0, 0, 1, 1])])

## 4. 定义神经网络模型

In [8]:
# 全连接神经网络
class Model(torch.nn.Module):
    #初始化
    def __init__(self):
        super().__init__()
        
        #定义网络结构
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(in_features=2, out_features=32),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=32, out_features=32),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=32, out_features=2),
            torch.nn.Softmax(dim=1)
        )
        
    # 定义神经网络计算过程
    def forward(self, x):
        return self.fc(x)
    
model = Model()

model(torch.randn(8,2)).shape

torch.Size([8, 2])

## 5.训练模型

In [11]:
def train():
    #定义优化器,根据梯度更新参数
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    
    #定义损失函数
    loss_func = torch.nn.CrossEntropyLoss()
    
    #开始训练，启动模型的训练模式，启动dropout等
    model.train()
    
    #全量数据训练100轮
    for epoch in range(100):
        
        #按批次遍历loader中的数据
        for i, (x,y) in enumerate(loader):
            
            # 模型计算
            out = model(x)
            
            # 计算损失
            loss = loss_func(out, y)
            
            #根据损失，计算梯度
            loss.backward()
            
            # 根据损失，更新参数
            optimizer.step()
            
            # 清空梯度,准备下一轮训练
            optimizer.zero_grad()
        
        if epoch % 20 == 0:
            # 计算准确率
            acc = (out.argmax(dim=1) == y).sum().item() / len(y)
            print(f'epoch:{epoch}, loss:{loss.item():.4f}, acc:{acc:.4f}')
            
    # 保存模型到磁盘
    torch.save(model.state_dict(), 'model.pth')
    
train()

epoch:0, loss:0.4264, acc:1.0000
epoch:20, loss:0.3234, acc:1.0000
epoch:40, loss:0.3236, acc:1.0000
epoch:60, loss:0.3133, acc:1.0000
epoch:80, loss:0.3487, acc:1.0000


## 6. 测试模型

In [12]:
# 测试
# 注释的表明不计算梯度，节省计算资源
@torch.no_grad()
def test():
    # 加载模型
    model.load_state_dict(torch.load('model.pth'))
    
    # 启动测试模式，不启动dropout等
    model.eval()
    
    # 生成测试数据
    x, y = next(iter(loader))
    
    # 模型计算
    out = model(x).argmax(dim=1)
    
    print('预测结果：', out)
    print('真实结果：', y)
    print(out == y)
    
test()

预测结果： tensor([1, 1, 0, 1, 1, 1, 1, 1])
真实结果： tensor([1, 1, 0, 1, 1, 1, 1, 1])
tensor([True, True, True, True, True, True, True, True])
